# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df.hvplot.points("Lng", "Lat",
                                      geo = True,
                                      tiles = "OSM",
                                      size = "Humidity",
                                      color="City",
                                      alpha=.5)

# Display the map
humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 32) & 
                                   (city_data_df["Wind Speed"] < 5) & 
                                   (city_data_df["Cloudiness"] >10)]

# Drop any rows with null values
best_weather_df = best_weather_df.dropna()

# Display sample data
best_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230,
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230,
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231,
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
       "apiKey":geoapify_key,
       "limit":17
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
constitucion - nearest hotel: No hotel found
senneterre - nearest hotel: No hotel found
bambous virieux - nearest hotel: No hotel found
zangakatun - nearest hotel: No hotel found
stornoway - nearest hotel: No hotel found
chokurdakh - nearest hotel: No hotel found
drochia - nearest hotel: No hotel found
klaksvik - nearest 

port hardy - nearest hotel: No hotel found
sol-iletsk - nearest hotel: No hotel found
tabou - nearest hotel: No hotel found
srednekolymsk - nearest hotel: No hotel found
homer - nearest hotel: No hotel found
livingston - nearest hotel: No hotel found
lima - nearest hotel: No hotel found
christchurch - nearest hotel: No hotel found
kortkeros - nearest hotel: No hotel found
dicabisagan - nearest hotel: No hotel found
smithers - nearest hotel: No hotel found
peleduy - nearest hotel: No hotel found
mayumba - nearest hotel: No hotel found
port blair - nearest hotel: No hotel found
komsomolskiy - nearest hotel: No hotel found
qasigiannguit - nearest hotel: No hotel found
buraydah - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
moron - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
woodward - nearest hotel: No hotel found
georgiyevskoye - nearest hotel: No hotel found
kysyl-syr - nearest hotel: No hotel found
tashara - nearest hotel: No hotel fou

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,No hotel found
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230,No hotel found
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230,No hotel found
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231,No hotel found
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
569,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330,No hotel found
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563,No hotel found
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564,No hotel found
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat",
                                   geo = True,
                                   tiles = "OSM",
                                   size = "Humidity",
                                   color ="City", 
                                   alpha=0.5,
                                   hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)